In [1]:
from countminsketch import CountMinSketch
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.metrics import roc_auc_score

np.random.seed(10)
p= 10000
w = 4

waveform_data = pd.read_csv("sanitized_waveform.data",header = None)
waveform_data_exc = waveform_data.drop(21, 1)

seed_np = np.random.seed(1)
s = 1000.0                                                      #sample size
sqrt_s = np.sqrt(s)

f = np.random.uniform(1.0/sqrt_s,1.0 - (1.0/sqrt_s))            #locality parameter
print(f)

m=100

while(m>0):
    scores = []
    cm_sketch = CountMinSketch(p,w)
    d = waveform_data.shape[1] #dimension of the dataset
    random_vector = np.random.uniform(0,f,d-1)
    #print(random_vector)
    
    #np.random.seed(1)
    base =  np.maximum(2, 1/f)
    r = int(np.random.uniform(1.0+(0.5*math.log(s,base)),math.log(s,base)))
    
    #print waveform_data_exc.shape[1]
    waveform_randomly = waveform_data_exc.sample(n=r,axis = 1)
    #print(waveform_randomly.head())
    #print waveform_randomly.shape
    
    sample_data = waveform_data_exc.sample(n = int(s))
    
    indexes_sample = sample_data.index
    
    included_cols = waveform_randomly.columns

    col_max_whole = np.max(sample_data,axis = 0)
    col_min_whole = np.min(sample_data,axis = 0)

    Y_cap = (sample_data - col_min_whole)/(col_max_whole - col_min_whole)

    #print included_cols
    for col in Y_cap:
        if col not in included_cols:
            Y_cap[col] = -1
        else:
            Y_cap[col] = np.floor((Y_cap[col] + random_vector[col] )/ f)
    #print(Y_cap.head())
    #print Y_cap.shape
    
    data_tuples = [tuple(x) for x in Y_cap.to_records(index=False)]
    #print(data_tuples[0])
    for i in data_tuples:
        cm_sketch.add(i)
        
    Y_whole = waveform_data_exc
    norm_whole_data = (Y_whole - col_min_whole) / (col_max_whole - col_min_whole)
    
    for col in Y_whole:
        if col not in included_cols:
            norm_whole_data[col] = -1
        else:
            norm_whole_data[col] = np.floor(norm_whole_data[col] + random_vector[col] / f)
        
    whole_data_tuples = [tuple(x) for x in norm_whole_data.to_records(index=False)]
    #print len(whole_data_tuples[0])
    #print len(whole_data_tuples)
    for i in whole_data_tuples:
        cm_sketch.add(i)
        
    sample_data_tuples = [tuple(x) for x in Y_cap.to_records(index=False)]
    #print (sample_data_tuples[0])
    #print (whole_data_tuples[0])
    #print sample_data_tuples
    #(-1, -1, -1, -1, -1, -1, -1, 0.0, 1.0, 1.0, 1.0, -1, -1, 1.0, -1, -1, -1, -1, -1, -1, 2.0)
    #print whole_data_tuples[3288]
    for i in whole_data_tuples:
        if(i in data_tuples):
            scores.append(np.math.log(cm_sketch.query(i),2)+1)
        else:
            scores.append(np.math.log(cm_sketch.query(i),2))
            
    y_true = waveform_data[21]
    y_true = y_true.values.tolist()
    for i in range(0, len(y_true)):
        if y_true[i] == 2:
            y_true[i] = 1
    roc = roc_auc_score(y_true, scores)
    print roc
    m -=1

0.422269993919
0.44197189596
0.690764553878
0.564737862608
0.403726182024
0.384432134761
0.393225909921
0.493740742209
0.489649294155
0.532122507379
0.63499976574
0.73935016164
0.456085184291
0.591768810209
0.370674742043
0.412321736843
0.586621388335
0.565640666165
0.394637779356
0.522139410168
0.561977193849
0.550001982203
0.668542251567
0.573821760269
0.64012195957
0.536660852203
0.435503605808
0.469421809283
0.557720862511
0.615495785115
0.451035971586
0.419726167608
0.481575419236
0.548904562311
0.567577819504
0.227278542828
0.556491896392
0.54762784311
0.485148791397
0.607147104722
0.469795724928
0.405306538748
0.575257055743
0.566138019022
0.592724772858
0.725267507361
0.602521362747
0.557035200329
0.616725652235
0.597583874235
0.511720228206
0.451477462347
0.683058287592
0.555505299691
0.542564214381
0.790513534845
0.580485567757
0.548638766853
0.606898428293
0.512173431987
0.685454050723
0.556735166811
0.604845045753
0.477414594063
0.524615362437
0.565642468168
0.563863891101
